In [1]:
import optuna
import tensorflow as tf


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import optuna


# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Add a channel dimension (for grayscale images)
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

# Define the objective function for Optuna
def objective(trial):
    

    # Define the model architecture
    model = tf.keras.Sequential()
    
    # Add the convolutional layers
    model.add(layers.Conv2D(
        trial.suggest_int("conv1_filters", 16, 64, log=True),
        (3, 3),
        activation="relu",
        input_shape=(28, 28, 1)
    ))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(
        trial.suggest_int("conv2_filters", 32, 128, log=True),
        (3, 3),
        activation="relu"
    ))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    # Add the dense layers
    model.add(layers.Dense(
        trial.suggest_int("dense_units", 32, 128, log=True),
        activation="relu"
    ))

    model.add(layers.Dense(10, activation="softmax"))

    # Compile the model
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    # Train the model
    model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=1)

    # Evaluate the model
    _, accuracy = model.evaluate(x_test, y_test)

    # Return the negative accuracy (as Optuna tries to minimize the objective)
    return -accuracy

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_model = tf.keras.Sequential()
best_model.add(layers.Conv2D(
    best_params["conv1_filters"],
    (3, 3),
    activation="relu",
    input_shape=(28, 28, 1)
))
best_model.add(layers.MaxPooling2D((2, 2)))
best_model.add(layers.Conv2D(
    best_params["conv2_filters"],
    (3, 3),
    activation="relu"
))
best_model.add(layers.MaxPooling2D((2, 2)))
best_model.add(layers.Flatten())
best_model.add(layers.Dense(
    best_params["dense_units"],
    activation="relu"
))
best_model.add(layers.Dense(10, activation="softmax"))

best_model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

best_model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Evaluate
_, accuracy = model.evaluate(x_test, y_test)

In [7]:
import importlib.util
name = ___ # name user
# Specify the absolute path to the CustomCNN.py file
custom_cnn_path = f'C:/Users/{name}/Documents/Git/AI-WORK/Advanced-Ai/CNN/CustomCNN.py'

# Load the CustomCNN module
spec = importlib.util.spec_from_file_location('CustomCNN', custom_cnn_path)
CustomCNN = importlib.util.module_from_spec(spec)
spec.loader.exec_module(CustomCNN)

liste  = [3,2,"gelu",3,3,"AveragePooling1D"]
a = CustomCNN.CNN_Layer(*liste)

In [ ]:
def Optuna_Listelements(name_layer,liste,key,trial):
    if type(liste[0])==int:
        return trial.suggest_loguniform(f"{name_layer}+{key}", liste[0], liste[1]) 
    elif type(liste[0])==str:
        return trial.suggest_categorical(f"{name_layer}+{key}", liste)

In [ ]:
for k in dictionnary:
    dictionnary[k]

In [40]:
def Optimizing_Layer(trial,name_layer,layer):
    filters = trial.suggest_loguniform(f{name_layer}, 8, 256) 


{'hyperparameter_filters': ListWrapper([8, 256, 8]),
 'hyperparameter_kernel_size': ListWrapper([1, 10, 1]),
 'hyperparameter_pool_size': ListWrapper([1, 10, 1]),
 'hyperparameter_strides': ListWrapper([1, 10, 1]),
 'hyperparameter_pooling_layer_name': ListWrapper(['MetaPoolingLayer', 'AveragePooling2D', 'MaxPooling2D', 'AveragePooling1D', 'MaxPooling1D'])}

In [42]:
liste =a.get_hyperparameter()["hyperparameter_filters"]